<a href="https://colab.research.google.com/github/Mike030668/Rotation_tran_Text2Image_2_task_Tex2Video/blob/main/notebooks/test_trainloop_R_spliter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Mike030668/Rotation_tran_Text2Image_2_task_Tex2Video.git -q

In [1]:
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAIN_DIR =  "/content/Rotation_tran_Text2Image_2_task_Tex2Video"
import sys
sys.path.append(MAIN_DIR)

print(f'device is {DEVICE}\n')
torch.cuda.memory_summary(device=DEVICE, abbreviated=False)

device is cuda



'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |

# seed freeze

In [2]:
import numpy as np
from torch import nn
import random
# отрисовка прохождения цикла
from tqdm.notebook import tqdm
import time

from utills.step_points import ComputeDiffPoints, Shuff_Reshuff
from utills.rotations import RotationVectors
from models.R_baseline import R_SpliterSimple
from utills.clearing import flush_memory
from utills.utills import optimizer_to
from utills.download import get_gd_file

In [3]:
seed = 10

random.seed(seed)
np.random.seed(seed)

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Load tensor data

In [ ]:
!pip install --upgrade --no-cache-dir gdown -q
!gdown --id "1msbazrYX8KgxHzVaU7WwGOFUl5GM3xjk"
!unzip /content/bothway_embed_data_min_300.zip

In [5]:
all_data = torch.load("/content/bothway_embed_data_min_300.pth")

# Both ways steps base spliter

In [6]:
model = R_SpliterSimple(max_delta_time = 100,
                        emb_dim = 1280,
                        ways = 2
                        )


In [7]:
total_params = sum(p.numel() for p in model.parameters())
print(f"total_params {total_params}\n")

#If you want to calculate only the trainable parameters:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"trainable_params {trainable_params}")


total_params 1791657

trainable_params 1791657


# Trainer

In [20]:
BASE_PATH = "/content/drive/...."
PATH_dir = BASE_PATH + '/....'

EPOCHS = 10
max_batch = 15 # max farmes selected to learn

# inite optimizer and scheduler
LR_RATE = 1e-03
EXP_RATE = 0.9
GAMMA = 0.9
optimizer = torch.optim.AdamW(model.parameters(), lr=LR_RATE)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

next_train =  False

# load previos state
if next_train:
    update_lr = False
    last_state_model = '/model_loss_bothway_ep_77_diffsteps_shuffle_directions.pt'
    checkpoint = torch.load(PATH_dir+ last_state_model)
    model.load_state_dict(checkpoint['model_state_dict'])
    last_save = checkpoint['saved_model']
    last_epoch = checkpoint['epoch']
    last_lr = checkpoint["last_history"]["lr"][-1] #
    best_loss = checkpoint['loss']
    best_loss_mse = checkpoint['mse_loss']
    best_loss_cos = checkpoint['cos_loss']
    best_acc = checkpoint['acc']
    best_base_loss = checkpoint['base_loss']
    history_train = checkpoint['all_history']

    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    for g in optimizer.param_groups:
        g['lr'] = LR_RATE  if update_lr else last_lr
    scheduler.last_epoch = last_epoch

    best_eph = {"loss" : last_epoch,
                "cos" : last_epoch,
                "mse" : last_epoch,
                "acc" : last_epoch,
                }


# state for start
else:
    last_save = ''
    last_epoch = 0
    last_lr = LR_RATE
    best_loss = 1000000
    best_loss_mse = 1000000
    best_loss_cos = 1000000
    best_eph = 0
    best_acc = 0

    history_train = {"loss" : [],
                    "loss_cos" : [],
                    "loss_mse" : [],
                    "acc" : [],
                    "lr" : [],
                    "base_loss" : []
                     }

    best_eph = {"loss" : 0,
                "cos" : 0,
                "mse" : 0,
                "acc" : 0,
                }


# correct save, train and lerning rate
JDUN_END = 3
JDUN_TRAIN = 3


# details for train
add_back_train = True
suff_direct =  True            # suffle or not directions in batch
add_rote_train = True
add_diff_train = True


treshold_step = 0

use_on_epoch = 0.05 # float 0< and <1
# which each ep save best checkpoint
friq_save_checkpoint = 15

# https://blog.paperspace.com/pytorch-loss-functions/
cos_loss = nn.CosineEmbeddingLoss(reduction='none').to(DEVICE)
mse_loss = nn.MSELoss(reduction='none').to(DEVICE)
target = (-1) *torch.ones(1280).to(DEVICE)

# to(DEVICE)
model.to(DEVICE)
optimizer_to(optimizer, DEVICE)

In [21]:
# inite ComputeDiffPoints class
cdp = ComputeDiffPoints(treshold = treshold_step)

# inite rotation class
RV = RotationVectors()

# paths to save temory weights
PATH_model_loss = PATH_dir+'/last_best_loss.pt'
PATH_model_cos = PATH_dir+'/last_best_cos.pt'
PATH_model_mse = PATH_dir+'/last_best_mse.pt'
PATH_model_acc = PATH_dir+'/last_best_acc.pt'

dict_model_paths = {"loss" : PATH_model_loss,
                    "cos" : PATH_model_cos,
                    "mse" : PATH_model_mse,
                    "acc" : PATH_model_acc,
                    "temp": PATH_dir + "/temp_checkpoints"
                    }


hist = {"loss" : [],
        "loss_cos" : [],
        "loss_mse" : [],
        "acc" : [],
        "base_loss" : [],
        "lr" : []
}

logs = {"bad_movi" : [],
        "empty" : [],
        "naninf_loss" : [],
        "naninf_mse" : [],
        "naninf_acc" : [],
        "naninf_cos" : [],
}

# init waitings jdun
wait_train = 0
wait2end = 0

for epoch  in tqdm(range(EPOCHS), unit ="EPOHS", desc ="Пробегаемся по всем эпохам"):
    # inite
    flush_memory()
    text = ''
    save_model = 0
    eph_loss = 0
    eph_loss_cos = 0
    eph_loss_mse = 0
    eph_cos_acc = 0
    eph_base_loss = 0
    cos_acc = 0
    cur_lr = optimizer.param_groups[0]['lr']
    diff_loss_weight = 0

    random.shuffle(all_data)  # shuffle  data each epoch
    take = int(len(all_data)*use_on_epoch)
    take_data = all_data[:take]  # take shuffle part data each epoch
    #print(f"Take for train {len(take_data)} movies")
    for  id_m, data  in tqdm(enumerate(take_data), unit = f" movi ",
                                        desc ="Пробегаемся по всем фильмам"):
        flush_memory()
        # get next frame embbedings
        id_movi, ids_frame = data['id_movi'], data['ids_frames']
        text_hid_state, unclip_embed  = data['last_hidden_state'], data['text_embed']
        image_embeds = data['img_embeds']

        # control long movi
        qty_frames = len(image_embeds)
        d_batch = max_batch-1
        if qty_frames and id_movi:
            if qty_frames <= max_batch:
                d_batch = qty_frames-1

            # take random ids in ids_frame[1:] because ids_frame[0] will first
            rand_ids = [0] + list(np.random.choice(np.arange(1, qty_frames-1),
                                                    d_batch-1, replace=False))
            rand_ids = rand_ids + [qty_frames-1]
            # select rand_ids labels and image_embeds
            image_embeds = torch.concat([image_embeds[i] for i in rand_ids]).unsqueeze(dim=1)
            labels = np.array([ids_frame[i] for i in rand_ids])

            # place labels to class points
            cdp.time_labels = labels
            config_norm, config_back = cdp.getpoints_train()
            all_points = cdp.points
            if add_back_train:
                all_points = np.append(all_points, cdp.back_points)

            # intit class for shufflee
            b_srs = Shuff_Reshuff(len(all_points))
            if suff_direct:
                bs_idx = b_srs.shuffle() # shuffleed indexees
                bu_idx = b_srs.unshuffle() # indexees for re_shuffle
            else:
                bs_idx = b_srs.idx_base # shuffleed indexees
                bu_idx = b_srs.idx_base # indexees for re_shuffle

            ######### compute enters and states for model and losses for base ways

            # collect time directions tensors
            time_base = torch.tensor(all_points).unsqueeze(1)
            directions = [torch.zeros_like(time_base[:d_batch])]
            if add_back_train:
                directions.append(torch.ones_like(time_base[d_batch:]))
            directions = torch.concat(directions)
            time_base = time_base.to(DEVICE)
            directions = directions.to(DEVICE)

            # collect text tensors
            text_hid_states = [text_hid_state for _ in range(d_batch)]
            if add_back_train:
                text_hid_states.extend([text_hid_state for _ in range(d_batch)])
            text_hid_states =  torch.concat(text_hid_states).to(DEVICE)

            # collect unclip tensors
            base_unclip_embs = [unclip_embed for _ in range(d_batch)]
            if add_back_train:
                base_unclip_embs.extend([unclip_embed for _ in range(d_batch)])
            base_unclip_embs = torch.concat(base_unclip_embs).unsqueeze(dim=1).to(DEVICE)

            # collect base_img_embs tensors
            base_img_embs = [image_embeds[0] for _ in range(d_batch)]
            if add_back_train:
                base_img_embs.extend([image_embeds[-1] for _ in range(d_batch)])
            base_img_embs = torch.concat(base_img_embs).unsqueeze(dim=1)

            # collect img_embs tensors
            img_embs = image_embeds[1:]

            if add_back_train:
                # collect back_img_embs tensors
                back_img_embs = torch.flip(image_embeds, [0,])[1:]
                # collect img_embs together tensors
                img_embs = torch.concat([img_embs, back_img_embs])
            ####################################################################

            # img difference
            diff_img_embs =  (base_img_embs[bs_idx].squeeze(dim=1).to(torch.float32) - img_embs[bs_idx].squeeze(dim=1).to(torch.float32)) #
            # check on static movi
            bad_movi = 0
            for k in range(diff_img_embs.shape[0]):
              if abs(diff_img_embs[k].sum()) ==0:
                bad_movi+=1

            if not bad_movi:
                # zero_grad optimizer
                optimizer.zero_grad()

                # base predict which can used in next step
                pred_unclip_embs = model(
                  text_hidden_states = text_hid_states[bs_idx].to(torch.float32), # shuffleed
                  prior_embeds = base_unclip_embs[bs_idx].to(torch.float32),
                  delta_time = time_base[bs_idx],
                  direction = directions[bs_idx]
                                    )

                # difference
                diff_unclip_embs = (base_unclip_embs[bs_idx].squeeze(dim=1).to(DEVICE).to(torch.float32) - pred_unclip_embs.squeeze(dim=1))

                # CosineEmbeddingLoss between difference
                movi_cos_loss = cos_loss(diff_unclip_embs.T,
                                        diff_img_embs.T.to(DEVICE), target).half()  #
                # control NAN INF in loss
                if torch.isnan(movi_cos_loss).sum() or torch.isinf(movi_cos_loss).sum():
                  print(f"\rMovi {id_movi} movi_cos_loss_base isnan or isinf")
                  logs["naninf_cos"].append(id_movi)
                  break

                # MSELoss between predict and
                movi_mse_loss = mse_loss(diff_unclip_embs.half(),
                                        diff_img_embs.half().to(DEVICE)).mean(0)#
                # control NAN INF in loss
                if torch.isnan(movi_mse_loss).sum() or torch.isinf(movi_mse_loss).sum():
                  print(f"\rMovi {id_movi} movi_mse_loss_base isnan or isinf")
                  logs["naninf_mse"].append(id_movi)
                  break

                # collect base_loss
                movi_loss_base = movi_cos_loss + movi_mse_loss # (0) for 1280
                eph_base_loss = movi_loss_base.mean().item()

                # CosineEmbeddingLoss between difference
                cos_acc = cos_loss(pred_unclip_embs.squeeze(1).T,
                                  img_embs[bs_idx].squeeze(1).T.to(DEVICE), (-1)*target).half()
                if epoch:
                    # compute coeficent for rote loss
                    rote_loss_weight = hist["acc"][-1]

                # control NAN INF in acc
                if torch.isnan(cos_acc).sum() or torch.isinf(cos_acc).sum():
                  print(f"\rMovi {id_movi} cos_acc_base isnan or isinf")
                  logs["naninf_acc"].append(id_movi)
                  break
                # collect cos_acc
                eph_cos_acc+= cos_acc.mean().item()

                # temp show losses for batch
                if id_m:
                  print(f"\rMovi {id_movi} step {id_m} base_way mse {eph_loss_mse/id_m:.5f} | cos {eph_loss_cos/id_m:.5f} | lr {cur_lr:.8f}", end="")

                # clear
                del(time_base, directions, diff_img_embs, diff_unclip_embs)
                flush_memory()

                #########  Rotation train steps ########################################
                to_rote, rote_norm, rote_back = 0, 0, 0
                if add_rote_train and epoch :
                    # Rotation train steps
                    rote_norm = len(config_norm['id_uclip_emb'])
                    rote_back = len(config_back['id_uclip_emb'])

                    # control size batch
                    if rote_norm > max_batch:
                      config_norm['id_uclip_emb'] = config_norm['id_uclip_emb'][:max_batch]
                      config_norm['id_img_emb_s'] = config_norm['id_img_emb_s'][:max_batch]
                      config_norm['id_img_delta'] = config_norm['id_img_delta'][:max_batch]
                      config_norm['delta'] = config_norm['delta'][:max_batch]
                      rote_norm = len(config_norm['id_uclip_emb'])

                    if rote_back > max_batch:
                      config_back['id_uclip_emb'] = config_back['id_uclip_emb'][:max_batch]
                      config_back['id_img_emb_s'] = config_back['id_img_emb_s'][:max_batch]
                      config_back['id_img_delta'] = config_back['id_img_delta'][:max_batch]
                      config_back['delta'] = config_back['delta'][:max_batch]
                      rote_back = len(config_back['id_uclip_emb'])

                    to_rote =  rote_norm + rote_back

                if add_rote_train and epoch and to_rote:
                    text_hid_states_2rt = []
                    unclip_embs_2rt = []
                    base_img_embs_2rt = []
                    image_embs_2rt = []
                    delta_2rt = []
                    direction_2rt = []

                    # intit class for shufflee again
                    srs = Shuff_Reshuff(to_rote)
                    if suff_direct:
                        s_idx = srs.shuffle()   # shuffleed indexees
                        u_idx = srs.unshuffle() # indexees for re_shuffle
                    else:
                        s_idx = srs.idx_base # shuffleed indexees
                        u_idx = srs.idx_base # indexees for re_shuffle

                    if rote_norm:
                        # collect norm steps
                        text_hid_states_2rt.append(torch.clone(text_hid_states[:d_batch])[config_norm['id_uclip_emb']])
                        unclip_embs_2rt.append(torch.clone(base_unclip_embs[:d_batch])[config_norm['id_uclip_emb']])
                        base_img_embs_2rt.append(torch.clone(img_embs[:d_batch])[config_norm['id_img_emb_s']])
                        image_embs_2rt.append(torch.clone(img_embs[:d_batch])[config_norm['id_img_delta']])
                        delta_2rt.append(torch.tensor(config_norm['delta']).unsqueeze(1))
                        direction_2rt.append(torch.zeros_like(delta_2rt[-1]))


                    if rote_back and add_back_train:
                        # collect back steps
                        text_hid_states_2rt.append(torch.clone(text_hid_states[d_batch:])[config_back['id_uclip_emb']])
                        unclip_embs_2rt.append(torch.clone(base_unclip_embs[d_batch:])[config_back['id_uclip_emb']])
                        base_img_embs_2rt.append(torch.clone(img_embs[d_batch:])[config_back['id_img_emb_s']])
                        image_embs_2rt.append(torch.clone(img_embs[d_batch:])[config_back['id_img_delta']])
                        delta_2rt.append(torch.tensor(config_back['delta']).unsqueeze(1))
                        direction_2rt.append(torch.ones_like(delta_2rt[-1]))


                    # shufle
                    text_hid_states_2rt = torch.concat(text_hid_states_2rt)[s_idx].to(DEVICE).to(torch.float32)
                    unclip_embs_2rt = torch.concat(unclip_embs_2rt)[s_idx].to(DEVICE).to(torch.float32)
                    base_img_embs_2rt = torch.concat(base_img_embs_2rt)[s_idx].to(DEVICE).to(torch.float32)
                    image_embs_2rt = torch.concat(image_embs_2rt)[s_idx].to(DEVICE).to(torch.float32)
                    delta_2rt = torch.concat(delta_2rt)[s_idx][s_idx].to(DEVICE)
                    direction_2rt = torch.concat(direction_2rt)[s_idx].to(DEVICE)

                    # get rotation vectors
                    R_marixes = RV.get_rotation_matrix(base_img_embs_2rt.squeeze(1),
                                                       image_embs_2rt.squeeze(1))
                    text_hid_states_2rt = torch.bmm(text_hid_states_2rt, R_marixes)
                    unclip_embs_2rt = torch.bmm(unclip_embs_2rt, R_marixes)

                    # rotation predict
                    pred_rote_embs = model(
                                  text_hidden_states = text_hid_states_2rt,
                                  prior_embeds = unclip_embs_2rt,
                                  delta_time = delta_2rt,
                                  direction = direction_2rt
                                                    )
                    # difference
                    diff_unclip_embeds = (unclip_embs_2rt.squeeze(dim=1) - pred_rote_embs.squeeze(dim=1))
                    diff_img_embs = (base_img_embs_2rt.squeeze(dim=1) - image_embs_2rt.squeeze(dim=1))

                    # CosineEmbeddingLoss between difference
                    movi_cos_loss += rote_loss_weight*cos_loss(diff_unclip_embeds.T,
                                                              diff_img_embs.T.to(DEVICE), target).half()  #
                    # control NAN INF in loss
                    if torch.isnan(movi_cos_loss).sum()or torch.isinf(movi_cos_loss).sum():
                      print(f"\rMovi {id_movi} movi_cos_loss_rote isnan or isinf")
                      logs["naninf_cos"].append(id_movi)
                      break

                    # MSELoss between predict and
                    movi_mse_loss += rote_loss_weight*mse_loss(diff_unclip_embeds.half(),
                                                              diff_img_embs.half().to(DEVICE)).mean(0) #
                    # control NAN INF in loss
                    if torch.isnan(movi_mse_loss).sum() or torch.isinf(movi_mse_loss).sum():
                      print(f"\rMovi {id_movi} movi_mse_loss_rote isnan or isinf")
                      logs["naninf_mse"].append(id_movi)
                      break

                    del(delta_2rt, direction_2rt, diff_img_embs, diff_unclip_embeds)
                    del(pred_rote_embs, text_hid_states_2rt, R_marixes)
                    del(unclip_embs_2rt,image_embs_2rt)


                #########  Diff train steps ########################################
                to_diff = 0
                # Diff train steps
                if add_diff_train:
                    config_diff_norm, config_diff_back = cdp.getpoints_diftrain()
                    diff_norm = len(config_diff_norm['id_uclip_emb'])
                    diff_back = len(config_diff_back['id_uclip_emb'])
                    to_diff = diff_norm + diff_back

                if add_diff_train and epoch>1 and to_diff:
                    diff_loss_weight = 1 - hist["base_loss"][-1]/hist["base_loss"][0]
                    if diff_loss_weight<0: diff_loss_weight = 0

                if add_diff_train and epoch>1 and to_diff and diff_loss_weight:
                    take_text_hid_states = []
                    take_base_unclip_embs = []
                    next_unclip_embs = []
                    next_base_img_embs = []
                    next_image_embs = []
                    next_delta = []
                    next_text_hid_states = []
                    next_direction = []

                    # un_shuffleed
                    pred_unclip_embs = torch.clone(pred_unclip_embs.detach().cpu())[bu_idx]

                    # intit class for shufflee again
                    srs = Shuff_Reshuff(to_diff)
                    if suff_direct:
                        s_idx = srs.shuffle() # shuffleed indexees
                        u_idx = srs.unshuffle() # indexees for re_shuffle
                    else:
                        s_idx = srs.idx_base # shuffleed indexees
                        u_idx = srs.idx_base # indexees for re_shuffle

                    if diff_norm:
                        # collect diff norm steps
                        take_base_unclip_embs.append(torch.clone(base_unclip_embs[:d_batch])[config_diff_norm['id_uclip_emb']])
                        take_text_hid_states.append(torch.clone(text_hid_states[:d_batch])[config_diff_norm['id_uclip_emb']])
                        next_unclip_embs.append(torch.clone(pred_unclip_embs[:d_batch])[config_diff_norm['id_uclip_emb']])
                        next_base_img_embs.append(torch.clone(img_embs[:d_batch])[config_diff_norm['id_img_emb_s']])
                        next_image_embs.append(torch.clone(img_embs[:d_batch])[config_diff_norm['id_img_delta']])
                        next_delta.append(torch.tensor(config_diff_norm['delta']).unsqueeze(1))
                        next_direction.append(torch.zeros_like(next_delta[-1]))

                    if diff_back:
                        # collect diff back steps
                        take_base_unclip_embs.append(torch.clone(base_unclip_embs[:d_batch])[config_diff_back['id_uclip_emb']])
                        take_text_hid_states.append(torch.clone(text_hid_states[d_batch:])[config_diff_back['id_uclip_emb']])
                        next_unclip_embs.append(torch.clone(pred_unclip_embs[d_batch:])[config_diff_back['id_uclip_emb']])
                        next_base_img_embs.append(torch.clone(img_embs[d_batch:])[config_diff_back['id_img_emb_s']])
                        next_image_embs.append(torch.clone(img_embs[d_batch:])[config_diff_back['id_img_delta']])
                        next_delta.append(torch.tensor(config_diff_back['delta']).unsqueeze(1))
                        next_direction.append(torch.ones_like(next_delta[-1]))


                    # collect torch.concat
                    take_base_unclip_embs = torch.concat(take_base_unclip_embs).to(DEVICE)
                    take_text_hid_states = torch.concat(take_text_hid_states).to(DEVICE)
                    next_unclip_embs = torch.concat(next_unclip_embs).to(DEVICE).to(torch.float32)
                    next_base_img_embs = torch.concat(next_base_img_embs).to(torch.float32)
                    next_image_embs = torch.concat(next_image_embs).to(torch.float32)
                    next_delta = torch.concat(next_delta)[s_idx].to(DEVICE)
                    next_direction = torch.concat(next_direction)[s_idx].to(DEVICE)

                    # get rotation vectors
                    R_marixes = RV.get_rotation_matrix(take_base_unclip_embs.squeeze(1).to(torch.float32),
                                                       next_unclip_embs.squeeze(1))
                    next_text_hid_states = torch.bmm(take_text_hid_states.to(torch.float32), R_marixes)

                    # dif predict from base predict
                    next_pred_unclip_embs = model(
                        text_hidden_states = next_text_hid_states[s_idx],
                        prior_embeds = next_unclip_embs[s_idx],
                        delta_time = next_delta,
                        direction = next_direction
                        )

                    # difference
                    diff_unclip_embeds = (next_unclip_embs[s_idx].squeeze(dim=1).to(DEVICE) - next_pred_unclip_embs.squeeze(dim=1))
                    diff_img_embs =  (next_base_img_embs[s_idx].squeeze(dim=1) - next_image_embs[s_idx].squeeze(dim=1))

                    # CosineEmbeddingLoss between difference
                    movi_cos_loss += rote_loss_weight*diff_loss_weight * cos_loss(diff_unclip_embeds.T,
                                                                diff_img_embs.T.to(DEVICE), target).half()  #
                    # control NAN INF in loss
                    if torch.isnan(movi_cos_loss).sum() or torch.isinf(movi_cos_loss).sum():
                      print(f"\rMovi {id_movi} movi_cos_loss_diff isnan or isinf")
                      logs["naninf_cos"].append(id_movi)
                      break

                    # MSELoss between predict and
                    movi_mse_loss += rote_loss_weight*diff_loss_weight * mse_loss(diff_unclip_embeds.half(),
                                                                diff_img_embs.half().to(DEVICE)).mean(0) #
                    # control NAN INF in loss
                    if torch.isnan(movi_mse_loss).sum() or torch.isinf(movi_mse_loss).sum():
                      print(f"\rMovi {id_movi} movi_mse_loss_diff isnan or isinf")
                      logs["naninf_mse"].append(id_movi)
                      break

                    del(diff_img_embs, diff_unclip_embeds)
                    del(next_image_embs, next_base_img_embs, next_delta, next_direction)
                    del(next_text_hid_states,  next_unclip_embs)
                    del(pred_unclip_embs, next_pred_unclip_embs)


                # collect loss
                movi_loss = movi_cos_loss + movi_mse_loss # (0) for 1280
                movi_loss.backward(torch.ones_like(movi_loss))
                print(f"\rMovi_loss.backward on {id_m} step ", end="")

                eph_loss_mse += movi_mse_loss.mean().item()
                eph_loss_cos += movi_cos_loss.mean().item()
                eph_loss += movi_loss.mean().item()

                # make clip_grad_norm model
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()

                # flush_memory
                del(movi_cos_loss, movi_mse_loss)
                del(image_embeds, base_img_embs, img_embs)
                del(text_hid_states, base_unclip_embs)
                flush_memory()

            else:

              logs["bad_movi"].append(id_movi)
              print(f'\rMovi {id_movi} has some static frames', end="")

        else:
          print(f'Movi {id_movi} empty')
          logs["empty"].append(id_movi)

    hist["lr"].append(cur_lr)
    good_steps = len(all_data)
    eph_cos_acc/=good_steps
    eph_loss/=good_steps
    eph_loss_mse/=good_steps
    eph_loss_cos/=good_steps
    eph_base_loss/=good_steps

    hist["loss"].append(eph_loss)
    hist["loss_mse"].append(eph_loss_mse)
    hist["loss_cos"].append(eph_loss_cos)
    hist["acc"].append(eph_cos_acc)
    hist["base_loss"].append(eph_base_loss)

    scheduler.step()

    if best_acc < hist["acc"][-1]:
        best_acc = hist["acc"][-1]
        last_save = "acc"
        text += f'- save_{last_save} '
        best_eph[last_save] = epoch
        torch.save(model.state_dict(), dict_model_paths[last_save])
        save_model += 1

    if best_loss_mse > hist["loss_mse"][-1]:
        best_loss_mse = hist["loss_mse"][-1]
        last_save = "mse"
        text += f'- save_{last_save} '
        best_eph[last_save] = epoch
        torch.save(model.state_dict(), dict_model_paths[last_save])
        save_model += 1

    if best_loss_cos > hist["loss_cos"][-1]:
        best_loss_cos = hist["loss_cos"][-1]
        last_save = "cos"
        text += f'- save_{last_save} '
        best_eph[last_save] = epoch
        torch.save(model.state_dict(), dict_model_paths[last_save])
        save_model += 1

    if best_loss > hist["loss"][-1]:
        best_loss = hist["loss"][-1]
        last_save = "loss"
        text += f'- save_{last_save} '
        best_eph[last_save] = epoch
        torch.save(model.state_dict(), dict_model_paths[last_save])
        save_model += 1

    # same pereodicaly station model to check
    if epoch and not epoch % friq_save_checkpoint:
        text += f'- save_{epoch}_ep_cpt '
        model_name = f"/tmp_cpt_{epoch}_a_{eph_cos_acc:.3f}_l_{eph_loss:.3f}_c_{eph_loss_cos:.3f}_m_{eph_loss_mse:.3f}.pt"
        torch.save(model.state_dict(), dict_model_paths["temp"] + model_name)

    if not save_model:
        wait_train+=1
        text += f'wait_{wait_train}'
    else:
      wait_train = 0
      wait2end = 0

    # finish training with loading best to predict
    if wait_train > JDUN_TRAIN or epoch == last_epoch + EPOCHS-1 or not epoch % friq_save_checkpoint:

        # load last best state
        model.load_state_dict(torch.load(dict_model_paths[last_save]))
        text += f' - load best_{last_save}_model {best_eph[last_save]} ep'

        if wait_train > JDUN_TRAIN:
            # update scheduler and optimizer
            GAMMA*=EXP_RATE
            LR_RATE =  hist["lr"][-1]*EXP_RATE
            optimizer = torch.optim.AdamW(model.parameters(), lr=LR_RATE)
            scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)
            scheduler.last_epoch = epoch
            wait_train = 0
            wait2end += 1


        if wait2end > JDUN_END or epoch == last_epoch + EPOCHS -1 or not epoch % friq_save_checkpoint:
            if not epoch % friq_save_checkpoint:
                text += f' - save best_{last_save}_model {best_eph[last_save]} ep checkpoint'
            if epoch:
                # take hist before best
                hist = {key: hist[key][:best_eph[last_save]] for key in hist.keys()}
            # update last history
            update_dict_hist = {key: history_train[key].extend(hist[key]) for key in history_train.keys()}
            # save checkpoint
            torch.save({
                'type_model' : model.__class__.__name__,
                'saved_model' : last_save,
                'epoch': best_eph[last_save],
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': best_loss,
                'cos_loss': best_loss_cos,
                'mse_loss': best_loss_mse,
                'acc': best_acc,
                'last_lr': hist["lr"][-1],
                'last_history': hist,
                'all_history': update_dict_hist,
                }, PATH_dir + f'/model_{last_save}_bothway_ep_{best_eph[last_save]}.pt')

            if wait2end > JDUN_END:
                print(f"\nStop train, don't good fitness already on {epoch} ep, save best model from {best_eph[last_save]} ep")
                break

            else:
                print(f'\nTrain is finished, saved the best model from {best_eph[last_save]}_ep')
    print(f'\rEp {epoch} all_loss {hist["loss"][-1]:.5f} | acc {hist["acc"][-1]:.5f} | mse_loss {hist["loss_mse"][-1]:.5f} | cos_loss {hist["loss_cos"][-1]:.5f} | lr {cur_lr:.5f} {text}\n')

Пробегаемся по всем эпохам:   0%|          | 0/10 [00:00<?, ?EPOHS/s]

Пробегаемся по всем фильмам: 0 movi  [00:00, ? movi /s]

Movi_loss.backward on 24 step 
Train is finished, saved the best model from 0_ep
Ep 0 all_loss 0.07623 | acc 0.04003 | mse_loss 0.06822 | cos_loss 0.00801 | lr 0.00100 - save_acc - save_mse - save_cos - save_loss  - load best_loss_model 0 ep - save best_loss_model 0 ep checkpoint



Пробегаемся по всем фильмам: 0 movi  [00:00, ? movi /s]

Ep 1 all_loss 0.07192 | acc 0.03703 | mse_loss 0.06444 | cos_loss 0.00747 | lr 0.00090 - save_mse - save_cos - save_loss 



...

Ep 7 all_loss 0.06476 | acc 0.03596 | mse_loss 0.05778 | cos_loss 0.00698 | lr 0.00048 - save_cos 



Пробегаемся по всем фильмам: 0 movi  [00:00, ? movi /s]

Ep 8 all_loss 0.06854 | acc 0.04014 | mse_loss 0.06056 | cos_loss 0.00797 | lr 0.00043 - save_acc 



Пробегаемся по всем фильмам: 0 movi  [00:00, ? movi /s]

Movi_loss.backward on 24 step 
Train is finished, saved the best model from 8_ep
Ep 9 all_loss 0.06476 | acc 0.03596 | mse_loss 0.05778 | cos_loss 0.00698 | lr 0.00039 wait_1 - load best_acc_model 8 ep

